In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
!pip install scipy
!pip install tenacity
!pip install tiktoken
!pip install termcolor 
!pip install openai
!pip install requests

  Obtaining dependency information for scipy from https://files.pythonhosted.org/packages/81/d7/d2537d51efb692d0c411e64267ba349e7668d40f5bc73cefe78ccd650dcd/scipy-1.11.3-cp311-cp311-win_amd64.whl.metadata
  Using cached scipy-1.11.3-cp311-cp311-win_amd64.whl.metadata (60 kB)
Using cached scipy-1.11.3-cp311-cp311-win_amd64.whl (44.1 MB)
  Obtaining dependency information for tiktoken from https://files.pythonhosted.org/packages/b8/eb/234646d9eefda8a500d0fd88b05bf625a90ed18054124349db26e558276e/tiktoken-0.5.1-cp311-cp311-win_amd64.whl.metadata
  Using cached tiktoken-0.5.1-cp311-cp311-win_amd64.whl.metadata (6.8 kB)
  Obtaining dependency information for regex>=2022.1.18 from https://files.pythonhosted.org/packages/b8/ad/3398312096118c4e62a5827664e52a04d5068e84d04142dd4a0da8a567ae/regex-2023.10.3-cp311-cp311-win_amd64.whl.metadata
  Using cached regex-2023.10.3-cp311-cp311-win_amd64.whl.metadata (41 kB)
Using cached tiktoken-0.5.1-cp311-cp311-win_amd64.whl (759 kB)
Using cached regex-202

In [4]:
from langchain.prompts import BaseChatPromptTemplate
#from langchain import SerpAPIWrapper, LLMChain
from langchain.chat_models import ChatOpenAI
from typing import List, Union
#from langchain.schema import AgentAction, AgentFinish, HumanMessage
#from langchain.memory import ConversationBufferWindowMemory
import openai
import re
import streamlit as st

from config import CHAT_MODEL, SYSTEM_PROMPT, CHATGPT_ANSWER_PROMPT

In [5]:
import json
import os
import requests
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored

#GPT_MODEL = "gpt-3.5-turbo-0613"

In [6]:
from dotenv import load_dotenv
load_dotenv(dotenv_path=".env.local")
openai.api_key = os.getenv("OPENAI_API_KEY", "")

@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, functions=None, function_call=None, model=GPT_MODEL):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + OPENAI_API_KEY,
    }
    json_data = {"model": model, "messages": messages}
    if functions is not None:
        json_data.update({"functions": functions})
    if function_call is not None:
        json_data.update({"function_call": function_call})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e


In [7]:
def pretty_print_conversation(messages):
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }
    
    for message in messages:
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and message.get("function_call"):
            print(colored(f"assistant: {message['function_call']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and not message.get("function_call"):
            print(colored(f"assistant: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "function":
            print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))


messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous or value is missing"})
messages.append({"role": "user", "content": "Onboard user nikhil.utane"})
chat_response = chat_completion_request(
    messages, 
)
assistant_message = chat_response.json()["choices"][0]["message"]
messages.append(assistant_message)
assistant_message


In [8]:
def generate_final_answer(question, rough_answer):

    chatgpt_answer_prepped = CHATGPT_ANSWER_PROMPT.format(
        QUESTION_HERE=question, ROUGH_ANSWER_HERE=rough_answer
    )    

    final_answer = openai.ChatCompletion.create(
        model=CHAT_MODEL,
        messages=[{"role": "system", "content": SYSTEM_PROMPT},
                  {"role": "user", "content": chatgpt_answer_prepped}],
        max_tokens=1000,
    )    
    # Response provided by GPT-3.5
    return final_answer["choices"][0]["message"]["content"]

In [9]:
def generate_all_answers(directory):
    for root, dirs, files in os.walk(directory):
        if "sample" in dirs:
            dirs.remove("sample")
        for dir in dirs:
            #print(os.path.join(root, dir))
            #print(dir)
            file_path =  os.path.join(root, dir) + '/'
            with open( file_path + '0-question.txt', 'r') as file:
                question = file.read()
                #print(question)
                
            with open( file_path + '1-rough-answer.txt', 'r') as file:
                rough_answer = file.read()
                #print(question)
                
            final_answer = generate_final_answer(question, rough_answer)    
            #print(final_answer)
            #print(rough_answer)
            with open( file_path + '2-chatgpt-answer.txt', 'w') as file:
                file.write(final_answer)
                print(file_path + '2-chatgpt-answer.txt' + " updated")

In [10]:
generate_all_answers("qna")
#print(openai.api_key)

qna\career_overview/2-chatgpt-answer.txt updated
qna\weakness/2-chatgpt-answer.txt updated
